# Simple Specification Analysis Example

This example demonstrates a simple analysis for various specs inside products. It uses the **Spec Service** to query, analyze and update the specs with the latest properties.

### Imports

Import Python Modules for executing the notebook. The **nisystemlink** library is used for communicating with various SystemLink Enterprise's endpoints. Scrapbook is used for running notebooks and recording data for the SystemLink Notebook Execution Service.

The SYSTEMLINK_API_KEY environment variable specifies an API key created for the user executing this notebook, which provides Role Based Access Control to the various SystemLink APIs called by this notebook. The API key will expire after 24 hours.
The SYSTEMLINK_HTTP_URI environment variable gives the base URL to the SystemLink instance executing this notebook.

In [ ]:
import os
import scrapbook as sb

from nisystemlink.clients.core import JupyterHttpConfiguration
from nisystemlink.clients.spec import SpecClient
from nisystemlink.clients.spec.models import QuerySpecificationsRequest, UpdateSpecificationsRequest

### Input Parameters

These are the parameters that the notebook expects to be passed in by SystemLink. For notebooks designed to be triggered by pressing the 'Analyze' button in SystemLink Specs grid inside product details page, they must tag the cell with 'parameters' and at minimum specify the following in the cell metadata using the JupyterLab Property Inspector (double gear icon):

```json
{
  "papermill": {
    "parameters": {
      "spec_ids": [],
      "product_id": ""
    }
  },
  "systemlink": {
    "namespaces": [],
    "parameters": [
      {
        "display_name": "spec_ids",
        "id": "spec_ids",
        "type": "string[]"
      },
      {
        "display_name": "product_id",
        "id": "product_id",
        "type": "string"
      }
    ],
    "version": 2
  },
  "tags": ["parameters"]
}
```

For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work)."


In [ ]:
spec_ids = []
product_id = ""

### Constants

In [ ]:
spec_client = SpecClient(JupyterHttpConfiguration())

In [ ]:
def __generate_spec_ids_filter():
    filter = ''
    for index, spec_id in enumerate(spec_ids):
        filter += f'specId == \"{spec_id}\"'
        if index < len(spec_ids) - 1:
            filter += " || "
    return filter   

def __batch_query_request(query_specs_request):
    data = []

    response = spec_client.query_specs(query_specs_request)
    # response = create_post_request(url, body)
    if response is not None and response.specs is not None:
        data.extend(response.specs)
    while response.continuation_token:
        query_specs_request.continuation_token = response.continuation_token
        response = spec_client.query_specs(query_specs_request)
        if response is not None and response.specs is not None:
            data.extend(response.specs)

    return data

def query_parametric_specs(product_id):
    spec_ids_filter = __generate_spec_ids_filter()
    query_specs_request = QuerySpecificationsRequest(
        product_ids=[product_id],
        filter=f"(({spec_ids_filter}) && (type == \"PARAMETRIC\"))",
        take=1000
    )
    specs = __batch_query_request(query_specs_request)

    return specs

In [ ]:
def analyze_spec(spec):
    # TODO: Implement Spec Analysis logic here

    spec.properties["Spec Analyzed"] = "True"

def update_specs(specs):
    update_specs_request = UpdateSpecificationsRequest(specs=specs)
    return spec_client.update_specs(update_specs_request)

def analyze_specs(specs):
    for spec in specs:
        analyze_spec(spec)

### Fetching, Analyzing and Updating Specs

In [ ]:
parametric_specs = query_parametric_specs(product_id)

analyze_specs(parametric_specs)
update_specs_response = update_specs(parametric_specs)

### Store the result information so that SystemLink can access it

SystemLink uses scrapbook to store result information from each notebook execution to display to the user in the Execution Details slide-out. Here we will displaying the details of updated specs."

In [ ]:
if  update_specs_response.updated_specs:
    sb.glue("Updated Specs: ", update_specs_response.updated_specs)
if  update_specs_response.failed_specs:
    sb.glue("Failed Specs: ", update_specs_response.failed_specs)

# Next Steps

1. Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as Specification Analysis.
1. Manually execute this notebook against the specs inside specs grid in product details page.
1. Go to spec details page to view the updated properties of the specs.